# Reduced Form Credit Risk Model - Jarrow & Lando
## Simple Implementation for MBA Students

**Based on:** Jarrow & Turnbull (1995) and Lando (1998)

**Key Idea:** Default is a "surprise" event modeled with a default intensity λ (lambda)


---
## Part 1: Setup


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("Libraries loaded successfully!")


---
## Part 2: The Core Model - Key Equations

### The three equations you need to know:

**1. Default Probability:**
$$P(\text{default by time T}) = 1 - e^{-\lambda T}$$

**2. Risky Bond Price:**
$$\text{Bond Price} = K \times e^{-(r + \lambda(1-\delta))T}$$

**3. Credit Spread:**
$$\text{Credit Spread} = -\frac{\ln(\text{Bond Price}/K)}{T} - r$$

Where:
- λ = default intensity (probability of default per year)
- K = face value ($100)
- r = risk-free rate
- δ = recovery rate (% you get back if default occurs)
- T = time to maturity


In [ ]:
# CORE FUNCTIONS - These implement the 3 key equations above

def default_probability(lambda_intensity, T):
    """Equation 1: Calculate probability of default by time T"""
    return 1 - np.exp(-lambda_intensity * T)


def risky_bond_price(K, r, lambda_intensity, delta, T):
    """Equation 2: Price of risky zero-coupon bond"""
    adjusted_rate = r + lambda_intensity * (1 - delta)
    return K * np.exp(-adjusted_rate * T)


def credit_spread(bond_price, K, r, T):
    """Equation 3: Credit spread over risk-free rate"""
    bond_yield = -np.log(bond_price / K) / T
    return bond_yield - r


---
## Part 3: Real Company Data

We use 5 real companies with different credit ratings and risk levels


In [ ]:
# REAL COMPANY DATA
companies = pd.DataFrame({
    'Company': ['Microsoft', 'Apple', 'JPMorgan', 'Ford', 'Tesla'],
    'Rating': ['AAA', 'AA+', 'A+', 'BB+', 'BB-'],
    'CDS_bps': [20, 25, 65, 280, 320],  # Credit Default Swap spread in basis points
    'Recovery': [0.40, 0.40, 0.40, 0.35, 0.35]  # Recovery rate (40% typical)
})

# Calculate implied default intensity from CDS spread
# Simple approximation: λ ≈ CDS / (1 - Recovery)
companies['Lambda'] = (companies['CDS_bps'] / 10000) / (1 - companies['Recovery'])

print("Real Company Credit Profiles:")
print(companies)
print("\nNote: Higher CDS spread → Higher Lambda → Higher default risk")


---
## Part 4: Example Calculation - Apple Inc.

Let's price a 5-year zero-coupon bond from Apple step-by-step


In [ ]:
# PARAMETERS
K = 100          # Face value
r = 0.03         # Risk-free rate (3%)
T = 5            # Maturity (5 years)

# Apple's parameters
apple = companies[companies['Company'] == 'Apple'].iloc[0]
lambda_apple = apple['Lambda']
delta_apple = apple['Recovery']

print("="*60)
print("PRICING APPLE 5-YEAR ZERO-COUPON BOND")
print("="*60)
print(f"\nGiven:")
print(f"  Face Value (K) = ${K}")
print(f"  Maturity (T) = {T} years")
print(f"  Risk-free rate (r) = {r*100:.1f}%")
print(f"  Apple's Lambda (λ) = {lambda_apple:.4f}")
print(f"  Recovery Rate (δ) = {delta_apple*100:.0f}%")

# STEP 1: Default Probability
default_prob = default_probability(lambda_apple, T)
print(f"\n--- STEP 1: Default Probability ---")
print(f"Formula: P(default) = 1 - exp(-λT)")
print(f"Calculation: 1 - exp(-{lambda_apple:.4f} × {T}) = {default_prob:.4f}")
print(f"Result: {default_prob*100:.2f}% chance of default in 5 years")

# STEP 2: Bond Price
bond_price = risky_bond_price(K, r, lambda_apple, delta_apple, T)
risk_free_price = K * np.exp(-r * T)

print(f"\n--- STEP 2: Bond Price ---")
print(f"Formula: Price = K × exp(-(r + λ(1-δ))T)")
print(f"Calculation: {K} × exp(-({r:.3f} + {lambda_apple:.4f}×{1-delta_apple:.2f})×{T})")
print(f"           = {K} × exp(-{(r + lambda_apple*(1-delta_apple))*T:.4f})")
print(f"Result: ${bond_price:.2f}")
print(f"\nComparison:")
print(f"  Risk-free bond price: ${risk_free_price:.2f}")
print(f"  Risky bond price: ${bond_price:.2f}")
print(f"  Discount for credit risk: ${risk_free_price - bond_price:.2f}")

# STEP 3: Credit Spread
spread = credit_spread(bond_price, K, r, T)
print(f"\n--- STEP 3: Credit Spread ---")
print(f"Formula: Spread = -ln(Price/K)/T - r")
print(f"Result: {spread*100:.2f}% = {spread*10000:.0f} basis points")
print(f"\nInterpretation: Apple's bond yields {spread*100:.2f}% more than risk-free")


---
## Part 5: Compare All Companies

Now let's price bonds for all 5 companies


In [ ]:
# Calculate bond prices for all companies
results = []

for _, company in companies.iterrows():
    # Calculate metrics
    def_prob = default_probability(company['Lambda'], T)
    bond_price = risky_bond_price(K, r, company['Lambda'], company['Recovery'], T)
    spread = credit_spread(bond_price, K, r, T)
    
    results.append({
        'Company': company['Company'],
        'Rating': company['Rating'],
        'Default Prob (%)': def_prob * 100,
        'Bond Price ($)': bond_price,
        'Credit Spread (bps)': spread * 10000
    })

results_df = pd.DataFrame(results)

print("\n" + "="*70)
print("5-YEAR ZERO-COUPON BOND PRICING RESULTS")
print("="*70)
print(f"Face Value: ${K} | Risk-free rate: {r*100:.1f}% | Maturity: {T} years")
print(f"Risk-free bond price: ${risk_free_price:.2f}")
print("\n" + results_df.to_string(index=False))

print("\n\nKey Insight:")
print("Lower credit rating → Higher default risk → Lower bond price → Higher spread")


---
## Part 6: Visualizations


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Chart 1: Bond Prices
ax1 = axes[0]
risk_free_prices = [risk_free_price] * len(results_df)
x = range(len(results_df))
width = 0.35

ax1.bar([i - width/2 for i in x], risk_free_prices, width, 
        label='Risk-Free', color='green', alpha=0.7)
ax1.bar([i + width/2 for i in x], results_df['Bond Price ($)'], width,
        label='Risky Bond', color='red', alpha=0.7)
ax1.set_xticks(x)
ax1.set_xticklabels(results_df['Company'], rotation=45, ha='right')
ax1.set_ylabel('Price ($)', fontweight='bold')
ax1.set_title('Bond Prices by Company', fontweight='bold', fontsize=12)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)
ax1.axhline(y=100, color='black', linestyle='--', alpha=0.3)

# Chart 2: Default Probabilities
ax2 = axes[1]
colors = ['darkgreen', 'green', 'orange', 'red', 'darkred']
bars = ax2.barh(results_df['Company'], results_df['Default Prob (%)'], color=colors, alpha=0.7)
ax2.set_xlabel('5-Year Default Probability (%)', fontweight='bold')
ax2.set_title('Default Risk by Company', fontweight='bold', fontsize=12)
ax2.grid(axis='x', alpha=0.3)
for i, (bar, val) in enumerate(zip(bars, results_df['Default Prob (%)'])):
    ax2.text(val + 0.3, i, f'{val:.2f}%', va='center')

# Chart 3: Credit Spreads
ax3 = axes[2]
ax3.bar(results_df['Company'], results_df['Credit Spread (bps)'], 
        color=colors, alpha=0.7)
ax3.set_ylabel('Credit Spread (basis points)', fontweight='bold')
ax3.set_xlabel('Company', fontweight='bold')
ax3.set_title('Credit Spreads', fontweight='bold', fontsize=12)
ax3.tick_params(axis='x', rotation=45)
ax3.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('reduced_form_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("Charts saved as: reduced_form_results.png")


---
## Part 7: Sensitivity Analysis

How does bond price change with different parameters?


In [ ]:
print("="*60)
print("SENSITIVITY ANALYSIS: JPMorgan Chase")
print("="*60)

jpmorgan = companies[companies['Company'] == 'JPMorgan'].iloc[0]

# Sensitivity to Recovery Rate
print("\n1. Impact of Recovery Rate:")
print("\nRecovery Rate | Bond Price | Credit Spread (bps)")
print("-" * 50)

recovery_rates = [0.20, 0.30, 0.40, 0.50, 0.60]
for delta in recovery_rates:
    price = risky_bond_price(K, r, jpmorgan['Lambda'], delta, T)
    spread = credit_spread(price, K, r, T)
    print(f"    {delta*100:5.0f}%     | ${price:8.2f}  |      {spread*10000:6.0f}")

print("\nInsight: Higher recovery rate → Higher bond price → Lower spread")

# Sensitivity to Maturity
print("\n\n2. Impact of Maturity (Term Structure):")
print("\nMaturity | Bond Price | Default Prob | Spread (bps)")
print("-" * 60)

maturities = [1, 2, 3, 5, 7, 10]
for T_mat in maturities:
    price = risky_bond_price(K, r, jpmorgan['Lambda'], jpmorgan['Recovery'], T_mat)
    def_prob = default_probability(jpmorgan['Lambda'], T_mat)
    spread = credit_spread(price, K, r, T_mat)
    print(f"  {T_mat:2d}Y    | ${price:8.2f}  |   {def_prob*100:6.2f}%   |    {spread*10000:6.0f}")

print("\nInsight: Longer maturity → Higher default probability → Lower price")


---
## Part 8: Simple Portfolio Example


In [ ]:
print("="*60)
print("PORTFOLIO ANALYSIS")
print("="*60)

# Create a simple portfolio
portfolio = pd.DataFrame({
    'Company': ['Microsoft', 'Apple', 'JPMorgan', 'Ford'],
    'Investment': [250000, 250000, 300000, 200000]  # Dollar amounts
})

# Merge with company data
portfolio = portfolio.merge(companies[['Company', 'Rating', 'Lambda', 'Recovery']], on='Company')

# Calculate for each position
portfolio['Num_Bonds'] = portfolio.apply(
    lambda row: row['Investment'] / risky_bond_price(K, r, row['Lambda'], row['Recovery'], T), 
    axis=1
)
portfolio['Default_Prob'] = portfolio['Lambda'].apply(lambda x: default_probability(x, T))
portfolio['Expected_Loss'] = portfolio['Investment'] * portfolio['Default_Prob'] * (1 - portfolio['Recovery'])

print(f"\nPortfolio Holdings (5-year zero-coupon bonds):")
print(portfolio[['Company', 'Rating', 'Investment', 'Default_Prob']].to_string(index=False))

print(f"\n\nPortfolio Summary:")
total_investment = portfolio['Investment'].sum()
weighted_default_prob = (portfolio['Default_Prob'] * portfolio['Investment']).sum() / total_investment
total_expected_loss = portfolio['Expected_Loss'].sum()

print(f"  Total Investment: ${total_investment:,.0f}")
print(f"  Weighted Avg Default Probability: {weighted_default_prob*100:.2f}%")
print(f"  Total Expected Loss: ${total_expected_loss:,.0f}")
print(f"  Expected Loss Rate: {total_expected_loss/total_investment*100:.2f}%")


---
## Part 9: Key Takeaways

### What We Learned:

1. **The Model:** Default is modeled as a surprise using intensity λ
- Higher λ = Higher default risk

2. **Three Key Equations:**
- Default Probability = 1 - e^(-λT)
- Bond Price = K × e^(-(r + λ(1-δ))T)
- Credit Spread = Extra yield for credit risk

3. **Recovery Rate Matters:**
- Higher recovery → Less loss in default → Higher bond price

4. **Real World Application:**
- Calibrate λ from CDS spreads
- Price corporate bonds
- Assess portfolio credit risk

### Why This Model (vs. Structural Models)?

- Uses **observable** market data (CDS spreads)
- Default is **realistic** (comes as surprise)
- **Easier to implement** and calibrate
- Better for **pricing and hedging** traded securities

### References:
- Jarrow & Turnbull (1995), Journal of Finance
- Lando (1998), Review of Derivatives Research
- Jarrow (2009), Annual Review of Financial Economics


---
## Exercises to Practice

**Exercise 1:** Change Apple's lambda to 0.01 and recalculate the bond price. What happens?

**Exercise 2:** Calculate the bond price for Tesla with a 3-year maturity instead of 5 years.

**Exercise 3:** Create a portfolio with only AAA and AA rated companies. Compare the expected loss to our mixed portfolio above.

**Exercise 4:** What recovery rate would make Ford's bond price equal to $80?